Handwritten Digit Recognition using MNIST Dataset

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Convolution2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import cv2 as cv

In [0]:
#load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
#checking out the shape of loaded dataset
print ("X_train:=====> {}".format(X_train.shape))
print ("y_train:=====> {}".format(y_train.shape))
print ("X_test: =====> {}".format(X_test.shape))
print ("y_test: =====> {}".format(y_test.shape))

X_train:=====> (60000, 28, 28)
y_train:=====> (60000,)
X_test: =====> (10000, 28, 28)
y_test: =====> (10000,)


In [0]:
#reshape dataset for our model
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

In [0]:

print ("X_train:=====> {}".format(X_train.shape))
print ("y_train:=====> {}".format(y_train.shape))
print ("X_test: =====> {}".format(X_test.shape))

print ("y_test: =====> {}".format(y_test.shape))

X_train:=====> (60000, 28, 28, 1)
y_train:=====> (60000,)
X_test: =====> (10000, 28, 28, 1)
y_test: =====> (10000,)


In [0]:
# one hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
#load train and test dataset
def load_dataset():
  #load dataset
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  #reshape dataset for our model
  X_train = X_train.reshape(60000, 28, 28, 1)
  X_test = X_test.reshape(10000, 28, 28, 1)
  # one hot encoding
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  return X_train, y_train, X_test, y_test

In [0]:
#scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

Building a Model

In [0]:
#define cnn model
def define_model():
  model = Sequential()
  model.add(Convolution2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D(pool_size=2))
  model.add(Convolution2D(32, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(128, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))

  #compile model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		X_train, y_train, X_test, y_test = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(X_test, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)
		# evaluate model
		_, acc = model.evaluate(X_test, y_test, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

In [108]:
# fit model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1204 - accuracy: 0.9657 - val_loss: 0.0424 - val_accuracy: 0.9860
Epoch 2/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1172 - accuracy: 0.9662 - val_loss: 0.0395 - val_accuracy: 0.9888
Epoch 3/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1165 - accuracy: 0.9664 - val_loss: 0.0395 - val_accuracy: 0.9875
Epoch 4/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1136 - accuracy: 0.9671 - val_loss: 0.0378 - val_accuracy: 0.9881
Epoch 5/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1120 - accuracy: 0.9676 - val_loss: 0.0372 - val_accuracy: 0.9881
Epoch 6/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.1060 - accuracy: 0.9691 - val_loss: 0.0363 - val_accuracy: 0.9885
Epoch 7/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1082 - accuracy: 0.9693 - val_loss: 0.0376 -

In [0]:
model.save('final_model.h5') #save model

In [0]:
def run_test_harness():
  #load dataset
  X_train, y_train, X_test, y_test = load_dataset()
  #prepare pixel data
  X_train, X_test = prep_pixels(X_train, X_test)
  #define model
  model = define_model()
  #fit model
  model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)
  #save model
  model.save('final_model.h5')

In [112]:
#entry point, run_test_harness
run_test_harness()

Epoch 1/10
1875/1875 [==============================] - 68s 37ms/step - loss: 0.2975 - accuracy: 0.9069 - val_loss: 0.0541 - val_accuracy: 0.9829
Epoch 2/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.1210 - accuracy: 0.9630 - val_loss: 0.0374 - val_accuracy: 0.9860
Epoch 3/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0981 - accuracy: 0.9704 - val_loss: 0.0325 - val_accuracy: 0.9890
Epoch 4/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0868 - accuracy: 0.9732 - val_loss: 0.0350 - val_accuracy: 0.9875
Epoch 5/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0791 - accuracy: 0.9754 - val_loss: 0.0265 - val_accuracy: 0.9910
Epoch 6/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0748 - accuracy: 0.9771 - val_loss: 0.0248 - val_accuracy: 0.9920
Epoch 7/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0694 - accuracy: 0.9792 - val_loss: 0.0216 -

In [0]:
# load and prepare the image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [0]:
# predict the class
digit = model.predict_classes(img)

In [143]:
img = load_image('49160.png')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [0]:
import tensorflow as tf
model = tf.keras.models.load_model('final_model.h5')

In [0]:
digit = model.predict_classes(img)

In [146]:
print(digit[0])

0
